In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import torch
import torch.nn as nn
import torch.optim as optim
import os

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.models import resnet18,googlenet
from sklearn.model_selection import train_test_split
from PIL import Image

In [27]:
def multiple_dir_files_to_df(dirs, label):
    df = pd.DataFrame()
    for idx,dir in enumerate(dirs):
        files = os.listdir(dir)
        files = [dir + '/' + file for file in files]
        df = pd.concat([df,pd.DataFrame({'file': files, 'label': label[idx]})])
    return df

common_path="Database/"

labels=["neutral","happy","sad","fear","angry","surprise","disgust"]

database=[common_path+label for label in labels]

df = multiple_dir_files_to_df(database,np.linspace(0,6,7))
X,y=df["file"].tolist(),df["label"].tolist()

In [28]:
# Définir un Dataset personnalisé
class CustomDataset(Dataset):
    def __init__(self, file_names, labels, transform=None):
        self.file_names = file_names
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        img_path = f'{self.file_names[idx]}' 
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return image, label


In [29]:
# Define the image transformations
transform = transforms.Compose([
    transforms.CenterCrop((500, 500)),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [30]:
class GoogleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.googlenet = googlenet(pretrained=True)
        self.googlenet.fc = nn.Linear(512, 7)
        self.googlenet.inception4d=nn.Identity()
        self.googlenet.inception4e=nn.Identity()
        self.googlenet.inception5a=nn.Identity()
        self.googlenet.inception5b=nn.Identity()
        self.googlenet.Linear=nn.Identity()

    def forward(self, x):
        return self.googlenet(x)

model= GoogleNet()
print(model)

c:\Users\Daito\miniconda3\envs\ContinualLearning\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Daito\miniconda3\envs\ContinualLearning\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


GoogleNet(
  (googlenet): GoogLeNet(
    (conv1): BasicConv2d(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv2): BasicConv2d(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv3): BasicConv2d(
      (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (inception3a): Inception(
      (branch1): BasicConv2d(
        (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn

In [31]:
# Définir le modèle ResNet
class ResNetModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = resnet18(pretrained=True)
        self.resnet.fc = nn.Linear(256, 7)
        self.resnet.layer4= nn.Identity()
        self.resnet.layer1[-1].bn2 = nn.Sequential(
            self.resnet.layer1[-1].bn2,
            nn.Dropout2d(p=0.15)
        )

        self.resnet.layer2[-1].bn2 = nn.Sequential(
            self.resnet.layer2[-1].bn2,
            nn.Dropout2d(p=0.15)
        )

        self.resnet.layer3[-1].bn2 = nn.Sequential(
            self.resnet.layer3[-1].bn2,
            nn.Dropout2d(p=0.15)
        )
        self.resnet.fc=nn.Identity()
        
    def forward(self, x):
        return self.resnet(x)

model= ResNetModel()
print(model)

c:\Users\Daito\miniconda3\envs\ContinualLearning\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNetModel(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_

In [38]:
# Créer les jeux de données et les chargeurs de données
t_dataset = CustomDataset(X, y, transform)
t_loader = DataLoader(t_dataset, batch_size=1, shuffle=False)

In [39]:
#Features extraction
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pretrained_model ="./googlenet_pretrained_7_emo_19.pth"
model = ResNetModel()  
model.load_state_dict(torch.load(pretrained_model, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')),strict=False)
model.eval()
features = []

for i, (x, y) in enumerate(t_loader):
    x = x.to(device)
    y = y.to(device)
    with torch.no_grad():
        output = model(x)
        features.append(output.cpu().numpy())


c:\Users\Daito\miniconda3\envs\ContinualLearning\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Daito\miniconda3\envs\ContinualLearning\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\Daito\AppData\Local\Temp\ipykernel_10228\3019966922.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.labels[idx], dtype=torch.float3

IndexError: index 10 is out of bounds for dimension 0 with size 10

In [37]:
print(features, len(features))
print(features[0].shape)

[array([[0.06898717, 0.04769126, 0.04066913, ..., 0.13482137, 0.13200359,
        0.11157239],
       [0.08847633, 0.04034041, 0.04538531, ..., 0.14032167, 0.09207691,
        0.05839076],
       [0.06149988, 0.03059203, 0.0600995 , ..., 0.13370708, 0.12369482,
        0.0975391 ],
       ...,
       [0.07165606, 0.02821405, 0.04296225, ..., 0.10762717, 0.10106985,
        0.07178736],
       [0.09047597, 0.03360276, 0.05206097, ..., 0.14261276, 0.12807561,
        0.1029763 ],
       [0.09301747, 0.0473428 , 0.0383539 , ..., 0.17037155, 0.12049063,
        0.08614142]], dtype=float32), array([[0.08041663, 0.04517825, 0.05939102, ..., 0.160543  , 0.13947183,
        0.09890022],
       [0.09413191, 0.03055649, 0.05412195, ..., 0.12594788, 0.14609617,
        0.11441696],
       [0.10790011, 0.0327044 , 0.03539018, ..., 0.16886003, 0.12147942,
        0.10881021],
       ...,
       [0.09776992, 0.05377175, 0.04383806, ..., 0.15450543, 0.09985338,
        0.08564284],
       [0.09738769